In [ ]:
import torch

# 1. (b). Attention

Define $X$, $W_Q$, $W_K$, and $W_V$. Remember that you need to convert them to tensors. Print the values.

### NOTE: Remember to round up your results maybe to 1 or 2 decimal places.

In [ ]:
# Initialisation

x = [
  [1, 0, 1, 0],
  [0, 2, 0, 2],
  [1, 1, 1, 1]
 ]

w_k = [
  [0, 0, 1],
  [1, 1, 0],
  [0, 1, 0],
  [1, 1, 0]
]

w_q = [
  [1, 0, 1],
  [1, 0, 0],
  [0, 0, 1],
  [0, 1, 1]
]

w_v = [
  [0, 2, 0],
  [0, 3, 0],
  [1, 0, 3],
  [1, 1, 0]
]

# TODO: Convert to tensors (use the tensor function from the torch library)

x = torch.tensor(x)
w_k = torch.tensor(w_k)
w_q = torch.tensor(w_q)
w_v = torch.tensor(w_v)

# TODO: Print the values

print(x)
print(w_k)
print(w_q)
print(w_v)



tensor([[1, 0, 1, 0],
        [0, 2, 0, 2],
        [1, 1, 1, 1]])
tensor([[0, 0, 1],
        [1, 1, 0],
        [0, 1, 0],
        [1, 1, 0]])
tensor([[1, 0, 1],
        [1, 0, 0],
        [0, 0, 1],
        [0, 1, 1]])
tensor([[0, 2, 0],
        [0, 3, 0],
        [1, 0, 3],
        [1, 1, 0]])


Calculate the values of $Q$, $K$, and $V$. Print the values.

In [ ]:
# TODO: Calculate Q, K, and V

k = torch.matmul(x , w_k)
q = torch.matmul(x , w_q)
v = torch.matmul(x , w_v)

# TODO: Print the values of Q, K, and V

print(k)
print(q)
print(v)



tensor([[0, 1, 1],
        [4, 4, 0],
        [2, 3, 1]])
tensor([[1, 0, 2],
        [2, 2, 2],
        [2, 1, 3]])
tensor([[1, 2, 3],
        [2, 8, 0],
        [2, 6, 3]])


Calculate the attention scores from the queries. Print the attention scores.

*Hint: Think which matrices you need for this.*

In [ ]:
# TODO: Calculate the attention scores matrix

attn_scores = torch.matmul(q , torch.transpose(k , 0 ,1))

# TODO: Print the matrix

print(attn_scores)

tensor([[ 2,  4,  4],
        [ 4, 16, 12],
        [ 4, 12, 10]])


Calculate the *softmax* for the attention scores. Print the value.

In [ ]:
from torch.nn.functional import softmax

# TODO: Calculate the softmax
attn_scores_softmax = softmax(attn_scores.type(torch.FloatTensor),dim=-1)

attn_scores_softmax = torch.round(attn_scores_softmax,decimals = 1)

# TODO: An extra step needs to be performed here. What could it be?
# Hint: Check the notebook again ;)

# TODO: Print the value
print(attn_scores_softmax)




tensor([[0.1000, 0.5000, 0.5000],
        [0.0000, 1.0000, 0.0000],
        [0.0000, 0.9000, 0.1000]])


Calculate the weighted values by multiplying the softmax attention scores with the values. Print the weighted values.

In [ ]:
# TODO: Calculate the weighted values (value matrix with attention scores)
v = v.type(torch.FloatTensor)
weighted_values = v[:,None] * attn_scores_softmax.T[:,:,None]

# TODO: Print the weighted values

print(weighted_values)

tensor([[[0.1000, 0.2000, 0.3000],
         [0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000]],

        [[1.0000, 4.0000, 0.0000],
         [2.0000, 8.0000, 0.0000],
         [1.8000, 7.2000, 0.0000]],

        [[1.0000, 3.0000, 1.5000],
         [0.0000, 0.0000, 0.0000],
         [0.2000, 0.6000, 0.3000]]])


Finally, sum up the weighted values to get your final result. Print the result.

In [ ]:
# TODO: Sum up the weighted values

outputs = torch.sum(weighted_values , dim=0)

# TODO: Print the final result

print(outputs)

# You may round up the values or leave them as they are.
# Did you get the same results as 1.(b)?



tensor([[2.1000, 7.2000, 1.8000],
        [2.0000, 8.0000, 0.0000],
        [2.0000, 7.8000, 0.3000]])


# 2. (b). Transfer Learning

The task that you will be working on here is to classify whether a given English sentence is grammatically correct or not. So, this is a task of text classification. You will be working with the CoLA dataset about which you will get know in one of the other slides below.

##*NOTE: Most of the code below is already completed for you. You need to complete the missing code in the cells. Also, do not miss any question that is asked in the text cells below. You need to complete the code in this Notebook and submit it. For the text answers, you may answer it here in the text cells, or you may answer them in the PDF along with other answers. If you do not submit the notebook, no marks will be awarded!*

First we will install some libraries that we need here.

In [ ]:
!pip install wget
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9674 sha256=9766adfcf68f8f4c1722532bf5cd1e886a383c543ea36127a73af917ffa7fbc1
  Stored in directory: /root/.cache/pip/wheels/bd/a8/c3/3cf2c14a1837a4e04bd98631724e81f33f462d86a1d895fae0
Successfully built wget
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 18.2 MB/s 
     |████████████████████████████████| 7.6 MB 59.0 MB/s 
     |████████████████████████████████| 182 kB 73.6 MB/s 


Next, we want to import all the important libraries we need.

In [ ]:
import os
import wget
import random
import numpy as np
import pandas as pd

import torch
from torch.utils.data import (TensorDataset, 
                              DataLoader, 
                              RandomSampler, 
                              SequentialSampler)

from transformers import (BertTokenizer, 
                          BertForSequenceClassification,
                          AdamW,
                          BertConfig,
                          get_linear_schedule_with_warmup)

# Don't worry, we are not using Tensorflow here.
# Just using a function from the Keras library
from tensorflow.keras.utils import pad_sequences

from sklearn.model_selection import train_test_split

We now assign the device on which we want our model to train. Preferbly a GPU, if not, then your CPU.

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


# TODO: Download the dataset

Here we download our data. We use the CoLA dataset.

The Corpus of Linguistic Acceptability (CoLA) dataset [here](https://nyu-mll.github.io/CoLA/cola_public_1.1.zip) consists of 10657 sentences from 23 linguistics publications, expertly annotated for acceptability (grammaticality) by their original authors.

In [ ]:
print('Downloading dataset...')

# TODO:
# Download the file
!wget https://nyu-mll.github.io/CoLA/cola_public_1.1.zip


# TODO:
# Unzip the dataset (if you have not already)

!unzip /content/cola_public_1.1.zip


--2022-12-10 11:21:22--  https://nyu-mll.github.io/CoLA/cola_public_1.1.zip
Resolving nyu-mll.github.io (nyu-mll.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to nyu-mll.github.io (nyu-mll.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 255330 (249K) [application/zip]
Saving to: ‘cola_public_1.1.zip.1’

cola_public_1.1.zip 100%[===================>] 249.35K  --.-KB/s    in 0.003s  

2022-12-10 11:21:22 (82.6 MB/s) - ‘cola_public_1.1.zip.1’ saved [255330/255330]

Archive:  /content/cola_public_1.1.zip
replace cola_public/README? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: cola_public/README      
  inflating: cola_public/tokenized/in_domain_dev.tsv  
  inflating: cola_public/tokenized/in_domain_train.tsv  
  inflating: cola_public/tokenized/out_of_domain_dev.tsv  
  inflating: cola_public/raw/in_domain_dev.tsv  
  inflating: cola_public/raw/in_domain_train.tsv  
  inflating: cola_public/raw/o

# TODO: Load the dataset

Here we load the data.

In [ ]:
# TODO:
# Load the dataset (in_domain_train.tsv) into a pandas dataframe.
# use the tab delimiter, and the columns ['sentence_source', 'label', 'label_notes', 'sentence']

df = pd.read_csv('cola_public/raw/in_domain_train.tsv', sep='\t', names = ['sentence_source', 'label', 'label_notes', 'sentence'])
df = df.reset_index(drop=True)


print('Number of training sentences: {:,}\n'.format(df.shape[0]))

# TODO:
# Display top 10 rows from the data
df.head(10)


Number of training sentences: 8,551



,sentence_source,label,label_notes,sentence
0,gj04,1,NaN,"Our friends won't buy this analysis, let alone..."
1,gj04,1,NaN,One more pseudo generalization and I'm giving up.
2,gj04,1,NaN,One more pseudo generalization or I'm giving up.
3,gj04,1,NaN,"The more we study verbs, the crazier they get."
4,gj04,1,NaN,Day by day the facts are getting murkier.
5,gj04,1,NaN,I'll fix you a drink.
6,gj04,1,NaN,Fred watered the plants flat.
7,gj04,1,NaN,Bill coughed his way out of the restaurant.
8,gj04,1,NaN,We're dancing the night away.
9,gj04,1,NaN,Herman hammered the metal flat.


In [ ]:
# TODO:
# Store the sentences and labels of the data in the variables

sentences = df.sentence.values
labels = df.label.values


# TODO: Text Preprocessing

Here we initialize the BERT tokenizer. You need to complete the code so that the tokenizer is loaded. Next, run the piece of code below and it should work properly if you have loaded the tokenizer without errors.

In [ ]:
MODEL_TOKENIZER_NAME = 'bert-base-uncased'

# TODO:
# Initialise the BertTokenizer here using the MODEL_TOKENIZER_NAME above and make sure the text is lowercase
# Refer: https://huggingface.co/docs/transformers/model_doc/bert#transformers.BertTokenizer

print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained(MODEL_TOKENIZER_NAME)

Loading BERT tokenizer...


Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Here we apply the tokenization to the data and encode the tokens to ids.


In [ ]:
input_ids = []
for sent in sentences:

    # TODO:
    # Use the tokenizer to tokenize each sentence 'sent'
    # Remember to add the special [CLS] and [SEP] tokens
    # If you are confused, refer the link in the previous cell

    encoded_sent = tokenizer.encode(text = sent, add_special_tokens = True)
    input_ids.append(encoded_sent)


Here we first find the max sequence length of our data and then apply padding to the data.


In [ ]:
# TODO:
# Calculate the max sequence length of the data (input_ids)

print('Max sentence length: ', max(len(x) for x in input_ids))

# Set the maximum sequence length.
# Although the max length is 47, we choose 64 as the max length here.
MAX_LEN = 64

print('\nPadding all sentences in data ...')

input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")

print('\nDone.')
type(input_ids)

Max sentence length:  47

Padding all sentences in data ...

Done.


numpy.ndarray

Here we create the attention mask for our sentences.

If a token ID is 0, then its padding. Set mask to 0.

If a token ID is not 0, then its a real token. Set mask to 1.

In [ ]:
# Create attention masks
attention_masks = []

for sent in input_ids:
    
    # TODO:
    # Calculate the attention mask here using the logic in the description above
      # for x in sent:
      #     if x!=0:
      #       att_mask.append(1)
      #     else:
      #       att_mask.append(0)
    att_mask = [1 if x != 0 else 0 for x in sent ] 

    # Store the attention mask for this sentence.
    attention_masks.append(att_mask)

# TODO: Data split

Here we split the dataset into train and validation sets. You may use the train_test_split() function from sklearn for this. Use any random_state value so that your results are reproducible. You may use the test set size of 10%.

Remember you also need to convert the values to tensors as this is pytorch.

In [ ]:
# TODO:
# Split the data into 90% training and 10% validation

train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, test_size=0.1)

# # TODO:
# Now split the masks in the same way

train_masks, validation_masks, _, _ = train_test_split(attention_masks, labels, test_size=0.1)



# TODO:
#Convert the values to the tensors here

train_inputs = torch.from_numpy(train_inputs)
validation_inputs = torch.from_numpy(validation_inputs)

train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)

train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)



Use the PyTorch DataLoader here to load the data into batches of BATCH_SIZE

In [ ]:
# PyTorch DataLoader needs to know our batch size for training

BATCH_SIZE = 32

# TODO:
# Create the DataLoader for our training set

train_data = TensorDataset(train_inputs, train_labels, train_masks)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader( train_data, batch_size = BATCH_SIZE, sampler=train_sampler)

# TODO:
# Now do the same for the validation set

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = RandomSampler(validation_data)
validation_dataloader = DataLoader(validation_data, batch_size=BATCH_SIZE, sampler = validation_sampler)

# TODO: Your Model

Here you load the pretrained model and load it into an instance of BertForSequenceClassification. This part is already done for you.

NOTE: This takes some time to download the pretrained model (~440 MB).

In [ ]:
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels = 2,
    output_attentions = False,
    output_hidden_states = False,
)

model.cuda()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

Next we prepare the Optimizer and the learning rate scheduler. AdamW is a class from the HuggingFace library. We use this as our optimizer.

In [ ]:
# TODO:
# Initialise the optimizer here using AdamW.
# Use the parameters of your model above, and the learning rate and epsilon values given below.
# Refer: Check the AdamW library from HuggingFace

LEARNING_RATE = 2e-5
EPSILON = 1e-8
optimizer = AdamW(model.parameters(), lr = LEARNING_RATE, eps = EPSILON)

epochs = 4

# Total number of training steps is number of batches * number of epochs.
total_steps = len(train_dataloader) * epochs

# TODO:
# Create the learning rate scheduler with the optimizer
# Use zero warmup steps and the total steps of training as inputs
# Check the get_linear_schedule_with_warmup function from HuggingFace

scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = 0, num_training_steps = total_steps)

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Here you will define the function to find the accuracy of your predictions.

In [ ]:
# TODO:
# Function to calculate the accuracy
# You need to find how many times predictions are equal to labels, and then divide it by number of labels

def flat_accuracy(preds, labels):
  
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    
    return  np.sum(pred_flat == labels_flat) / len(labels_flat)

Training and Validation phases. Complete the missing pieces of code, and ensure that the code runs properly with errors.

In [ ]:
# TODO:
# Set your own seed value for easy reproducibility of your results

seed_val = 56

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

loss_values = []

######### TRAINING PHASE ###########

for epoch_i in range(epochs):
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    total_loss = 0
    model.train()

    # For each batch
    for step, batch in enumerate(train_dataloader):

        # Progress logged every 50 batches
        if step % 50 == 0 and not step == 0:

            print('  Batch {:>5,}  of  {:>5,}'.format(step, len(train_dataloader)))

        # TODO: 
        # The batch variable contains input_ids, input_masks, and labels.
        # You need to assign each of them to the 'device' for training
        
        #print(batch[2])

        b_input_ids = batch[0].to(device)
        b_input_mask = batch[2].to(device)
        b_labels = batch[1].to(device)

#         # TODO:
#         # Here you need to clear any previously calculated gradients
        model.zero_grad()  
        

#         # TODO:
#         # Perform a forward pass here
#         # Remember that your model will need input_ids, input_mask, and labels
        
        #print( b_labels.size())
        outputs = model(b_input_ids, attention_mask= b_input_mask,  labels= b_labels)	
        
        # TODO:
        # The outputs variable above should now have the loss.
        # Extract the loss, and add it to the total_loss variable
        # NOTE THAT THESE ARE ALL TENSORS!!!
        #print(outputs['loss'])

        total_loss += outputs['loss'].item()

        # TODO:
        # Perform a backward pass here

        outputs['loss'].backward()
        


        # The below code clips the gradients to 1.0 to prevent 
        # the "exploding gradients" problem.
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # TODO:
        # Update the parameters of the model here
        
        optimizer.step()

        # TODO:
        # Update the learning rate here
        
        scheduler.step()

    # TODO:
    # Calculate the average loss over the training data
    
    avg_train_loss = total_loss / len(train_dataloader)
    loss_values.append(avg_train_loss)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
        
######### VALIDATION PHASE ###########

    print("")
    print("Running Validation...")

    model.eval()

    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    for batch in validation_dataloader:
        
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch

        with torch.no_grad():        

            # TODO:
            # Perform a forward here too, this time for validation

            outputs = model(b_input_ids, labels = b_labels, attention_mask = b_input_mask)
        
        #logits = outputs[0]
        logits = outputs[1] # i think it should outputs[1] as outputs[0} would give loss but we need logits to calculate validation accuracy for this run
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))

print("")
print("Training DONE.")


======== Epoch 1 / 4 ========
Training...
  Batch    50  of    241
  Batch   100  of    241
  Batch   150  of    241
  Batch   200  of    241

  Average training loss: 0.56

Running Validation...
  Accuracy: 0.80

======== Epoch 2 / 4 ========
Training...
  Batch    50  of    241
  Batch   100  of    241
  Batch   150  of    241
  Batch   200  of    241

  Average training loss: 0.40

Running Validation...
  Accuracy: 0.81

======== Epoch 3 / 4 ========
Training...
  Batch    50  of    241
  Batch   100  of    241
  Batch   150  of    241
  Batch   200  of    241

  Average training loss: 0.27

Running Validation...
  Accuracy: 0.83

======== Epoch 4 / 4 ========
Training...
  Batch    50  of    241
  Batch   100  of    241
  Batch   150  of    241
  Batch   200  of    241

  Average training loss: 0.20

Running Validation...
  Accuracy: 0.82

Training DONE.
